<a href="https://colab.research.google.com/github/PandemicPreventionInstitute/NGS-Capacity-map/blob/main/FIND_support_maps_scripts_12_11_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Updating FIND NGS support maps, while using NGS November underlying data

In [2]:
#import packages
import pandas as pd
import seaborn as sns
import numpy as np

In [79]:
#import November NGS map data (exported from github) 
github_url = "https://raw.githubusercontent.com/PandemicPreventionInstitute/NGS-Capacity-map/main/data/LMIC%20centered%20map/NGS_flourish_file_11.9.2021_TEST3.csv"
df_NGS_Nov = pd.read_csv (github_url)
#drop first column "Unamed"
df_NGS_Nov = df_NGS_Nov.iloc[0: , 1:]
df_NGS_Nov.head(3)

,geometry,country,code,region,who_testing_capacity,owid_testing_capacity,pop_100k,max_new_tests_cap_avg,cap_cum_tests,cum_tpr,dx_testing_capacity,ngs_capacity,sequencing_capacity,total_sequences,submission_count,total_viable_sequences,all_cases,percent_cases_sequenced,sequences_per_capita_per_100k,sars_cov_2_sequencing,cases_newly_reported_in_last_7_days_per_100000_population,world_bank_economies,archetype_full,archetype,label,dx_testing_capacity_clean,sequencing_capacity_clean,sars_cov_2_sequencing_clean
0,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Afghanistan,AFG,Eastern Mediterranean,1 - Reliable,0.0,389.28,10.65,1970.843609,0.203075,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,94,99,94.0,155764.0,0.06,0.24,1-499 sequences,0.63,Low income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences
1,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Albania,ALB,Europe,1 - Reliable,1.0,28.78,190.16,43816.504517,0.141791,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,42,42,42.0,177971.0,0.02,1.46,1-499 sequences,103.62,Upper middle income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences
2,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Algeria,DZA,Africa,1 - Reliable,1.0,438.51,4.46,525.764521,0.891131,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,72,72,60.0,205286.0,0.04,0.16,1-499 sequences,1.66,Lower middle income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences


In [80]:
#import dataset that Swapna sent RE: FIND projects
FIND_github = "https://raw.githubusercontent.com/PandemicPreventionInstitute/NGS-Capacity-map/main/data/FIND_projects.csv"
df_FIND = pd.read_csv (FIND_github)
df_FIND.head()

,Country,Region,WB_eco,Archetype,N_projects,N_orgs
0,Cameroon,Africa,Lower middle income,Leverage,5,3
1,Brazil,Americas,Upper middle income,Strengthen,12,7
2,Guinea,Africa,Low income,Test,2,1
3,Lebanon,Eastern Mediterranean,Upper middle income,Leverage,3,1
4,Bangladesh,South-East Asia,Lower middle income,Strengthen,8,6


In [81]:
#renaming and merging files
df_FIND.rename (columns = {'Country': 'country'}, inplace = True)
df_FIND.head(3)

,country,Region,WB_eco,Archetype,N_projects,N_orgs
0,Cameroon,Africa,Lower middle income,Leverage,5,3
1,Brazil,Americas,Upper middle income,Strengthen,12,7
2,Guinea,Africa,Low income,Test,2,1


In [82]:
#merge together dataset
FIND_Projects_Flourish = df_NGS_Nov.merge(df_FIND, how = 'left', on = 'country')
FIND_Projects_Flourish.head(3)

,geometry,country,code,region,who_testing_capacity,owid_testing_capacity,pop_100k,max_new_tests_cap_avg,cap_cum_tests,cum_tpr,dx_testing_capacity,ngs_capacity,sequencing_capacity,total_sequences,submission_count,total_viable_sequences,all_cases,percent_cases_sequenced,sequences_per_capita_per_100k,sars_cov_2_sequencing,cases_newly_reported_in_last_7_days_per_100000_population,world_bank_economies,archetype_full,archetype,label,dx_testing_capacity_clean,sequencing_capacity_clean,sars_cov_2_sequencing_clean,Region,WB_eco,Archetype,N_projects,N_orgs
0,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Afghanistan,AFG,Eastern Mediterranean,1 - Reliable,0.0,389.28,10.65,1970.843609,0.203075,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,94,99,94.0,155764.0,0.06,0.24,1-499 sequences,0.63,Low income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences,NaN,NaN,NaN,NaN,NaN
1,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Albania,ALB,Europe,1 - Reliable,1.0,28.78,190.16,43816.504517,0.141791,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,42,42,42.0,177971.0,0.02,1.46,1-499 sequences,103.62,Upper middle income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences,NaN,NaN,NaN,NaN,NaN
2,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Algeria,DZA,Africa,1 - Reliable,1.0,438.51,4.46,525.764521,0.891131,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,72,72,60.0,205286.0,0.04,0.16,1-499 sequences,1.66,Lower middle income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences,NaN,NaN,NaN,NaN,NaN


In [83]:
#confirm countrys have right # of projects reported
Cameroon_projects = FIND_Projects_Flourish.loc[(FIND_Projects_Flourish['country'] == 'Cameroon')]
Cameroon_projects.head(4)

,geometry,country,code,region,who_testing_capacity,owid_testing_capacity,pop_100k,max_new_tests_cap_avg,cap_cum_tests,cum_tpr,dx_testing_capacity,ngs_capacity,sequencing_capacity,total_sequences,submission_count,total_viable_sequences,all_cases,percent_cases_sequenced,sequences_per_capita_per_100k,sars_cov_2_sequencing,cases_newly_reported_in_last_7_days_per_100000_population,world_bank_economies,archetype_full,archetype,label,dx_testing_capacity_clean,sequencing_capacity_clean,sars_cov_2_sequencing_clean,Region,WB_eco,Archetype,N_projects,N_orgs
34,"{""type"":""Polygon"",""coordinates"":[[[16.189823,2...",Cameroon,CMR,Africa,1 - Reliable,1.0,265.46,41.15,7897.246289,0.047839,Reliable testing capacity,1.0,1 - 1-3 NGS facilities or equivalent,211,220,211.0,98402.0,0.21,0.78,1-499 sequences,11.31,Lower middle income,3 - Leverage,Leverage,– Leverage existing NGS capacity,Reliable testing capacity,1-3 NGS facilities or equivalent,1-499 sequences,Africa,Lower middle income,Leverage,5.0,3.0


In [84]:
#confirm countrys have right # of projects reported
Brazil_projects = FIND_Projects_Flourish.loc[(FIND_Projects_Flourish['country'] == 'Brazil')]
Brazil_projects.head(4)

,geometry,country,code,region,who_testing_capacity,owid_testing_capacity,pop_100k,max_new_tests_cap_avg,cap_cum_tests,cum_tpr,dx_testing_capacity,ngs_capacity,sequencing_capacity,total_sequences,submission_count,total_viable_sequences,all_cases,percent_cases_sequenced,sequences_per_capita_per_100k,sars_cov_2_sequencing,cases_newly_reported_in_last_7_days_per_100000_population,world_bank_economies,archetype_full,archetype,label,dx_testing_capacity_clean,sequencing_capacity_clean,sars_cov_2_sequencing_clean,Region,WB_eco,Archetype,N_projects,N_orgs
26,"{""type"":""MultiPolygon"",""coordinates"":[[[[-58.1...",Brazil,BRA,Americas,1 - Reliable,1.0,2125.59,160.1,11084.685664,0.919504,Reliable testing capacity,2.0,2 - 4+ NGS facilities or equivalent,48613,48645,47847.0,21494358.0,0.23,22.72,500+ sequences,42.82,Upper middle income,4 - Strengthen,Strengthen,– Build additional NGS capacity for scale-up,Reliable testing capacity,4+ NGS facilities or equivalent,500+ sequences,Americas,Upper middle income,Strengthen,12.0,7.0


In [85]:
#confirm countrys have right # of projects reported
Ethiopia_projects = FIND_Projects_Flourish.loc[(FIND_Projects_Flourish['country'] == 'Ethiopia')]
Ethiopia_projects.head(4)

,geometry,country,code,region,who_testing_capacity,owid_testing_capacity,pop_100k,max_new_tests_cap_avg,cap_cum_tests,cum_tpr,dx_testing_capacity,ngs_capacity,sequencing_capacity,total_sequences,submission_count,total_viable_sequences,all_cases,percent_cases_sequenced,sequences_per_capita_per_100k,sars_cov_2_sequencing,cases_newly_reported_in_last_7_days_per_100000_population,world_bank_economies,archetype_full,archetype,label,dx_testing_capacity_clean,sequencing_capacity_clean,sars_cov_2_sequencing_clean,Region,WB_eco,Archetype,N_projects,N_orgs
64,"{""type"":""Polygon"",""coordinates"":[[[34.070698,9...",Ethiopia,ETH,Africa,1 - Reliable,1.0,1149.64,16.85,3135.248426,0.099845,Reliable testing capacity,NaN,0 - 0 NGS facilities or equivalent,219,219,219.0,359247.0,0.06,0.19,1-499 sequences,4.73,Low income,2 - Connect,Connect,– Set-up sample referral networks or build NGS...,Reliable testing capacity,0 NGS facilities or equivalent,1-499 sequences,Africa,Low income,Connect,11.0,5.0


Adding lat/long to the df and make sure to only include variables of interests.

In [86]:
#reading in github lat/long datafile
latlong_url = "https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv"
lat_long_df = pd.read_csv(latlong_url)
lat_long_df.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""


In [87]:
#replacing "" and changing data-type
lat_long_df = lat_long_df.replace('"','', regex=True)
lat_long_df.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33,65
1,Albania,AL,ALB,8,41,20
2,Algeria,DZ,DZA,12,28,3
3,American Samoa,AS,ASM,16,-14.3333,-170
4,Andorra,AD,AND,20,42.5,1.6


In [88]:
#change datatypes
lat_long_df["Alpha-3 code"] = lat_long_df["Alpha-3 code"].astype(str)
lat_long_df["Latitude (average)"] = lat_long_df["Latitude (average)"].astype(float)
lat_long_df["Longitude (average)"] = lat_long_df["Longitude (average)"].astype(float)
print (lat_long_df.dtypes)
lat_long_df.head()

Country                 object
Alpha-2 code            object
Alpha-3 code            object
Numeric code            object
Latitude (average)     float64
Longitude (average)    float64
dtype: object


,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [89]:
#rename variables
lat_long_df.rename (columns = {'Alpha-3 code': 'code'}, inplace = True)
lat_long_df.rename (columns = {'Latitude (average)': 'latitude'}, inplace = True)
lat_long_df.rename (columns = {'Longitude (average)': 'longitude'}, inplace = True)
lat_long_df.rename (columns = {'Country': 'country'}, inplace = True)
lat_long_df.head()

,country,Alpha-2 code,code,Numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [90]:
#only keeping necessary variables from this dataset, for our observations
lat_long_df = lat_long_df[['country', 'code', "latitude", 'longitude']]
lat_long_df.head()
print (lat_long_df.dtypes)

country       object
code          object
latitude     float64
longitude    float64
dtype: object


In [91]:
FIND_df = FIND_Projects_Flourish[['code', "country", 'geometry', 'archetype', 'label', 
                                  'max_new_tests_cap_avg', 'sequencing_capacity_clean', 'total_sequences', 'WB_eco', 'N_projects', 'N_orgs' ]]

print (FIND_df.dtypes)
FIND_df.head()

code                          object
country                       object
geometry                      object
archetype                     object
label                         object
max_new_tests_cap_avg        float64
sequencing_capacity_clean     object
total_sequences                int64
WB_eco                        object
N_projects                   float64
N_orgs                       float64
dtype: object


,code,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs
0,AFG,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN
1,ALB,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN
2,DZA,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,NaN,NaN
3,ASM,American Samoa,"{""type"":""MultiPolygon"",""coordinates"":[[[[-170....",Test,– Increase diagnostic testing capacity,NaN,0 NGS facilities or equivalent,0,NaN,NaN,NaN
4,AND,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,NaN,NaN


In [92]:
#trying to round numbers...this did not work due to having NaN's
FIND_df[['N_projects','N_orgs']].astype(float).apply(np.round)
FIND_df.head()

,code,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs
0,AFG,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN
1,ALB,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN
2,DZA,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,NaN,NaN
3,ASM,American Samoa,"{""type"":""MultiPolygon"",""coordinates"":[[[[-170....",Test,– Increase diagnostic testing capacity,NaN,0 NGS facilities or equivalent,0,NaN,NaN,NaN
4,AND,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,NaN,NaN


In [93]:
#Confirm projects data is good
Projects_confirm = FIND_df.loc[(FIND_df['N_projects'] > 0)]
print ("N of countries w/projects:", Projects_confirm.country.nunique())
Projects_confirm.head()

N of countries w/projects: 59


,code,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs
8,ARG,Argentina,"{""type"":""MultiPolygon"",""coordinates"":[[[[-68.4...",Strengthen,– Build additional NGS capacity for scale-up,260.95,4+ NGS facilities or equivalent,8492,Upper middle income,2.0,2.0
11,AUS,Australia,"{""type"":""MultiPolygon"",""coordinates"":[[[[158.8...",High Income*,NaN,863.04,High Income*,34791,High income,1.0,1.0
15,BGD,Bangladesh,"{""type"":""MultiPolygon"",""coordinates"":[[[[89.07...",Strengthen,– Build additional NGS capacity for scale-up,25.41,4+ NGS facilities or equivalent,2940,Lower middle income,8.0,6.0
16,BRB,Barbados,"{""type"":""Polygon"",""coordinates"":[[[-59.42691,1...",High Income*,NaN,738.18,High Income*,78,High income,1.0,1.0
23,BOL,Bolivia,"{""type"":""Polygon"",""coordinates"":[[[-67.193904,...",Connect,– Set-up sample referral networks or build NGS...,103.68,0 NGS facilities or equivalent,152,Lower middle income,1.0,1.0


In [94]:
#merge together dataset
FIND_Regions_Points_Map = FIND_df.merge(lat_long_df, how = 'left', on = 'country')
FIND_Regions_Points_Map.head(3)

,code_x,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,code_y,latitude,longitude
0,AFG,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN,AFG,33.0,65.0
1,ALB,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN,ALB,41.0,20.0
2,DZA,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,NaN,NaN,DZA,28.0,3.0


In [95]:
print (FIND_Regions_Points_Map.shape)
print (FIND_df.shape)

(238, 14)
(238, 11)


In [96]:
#only keeping necessary metrics from this dataset, for our observations
FIND_Regions_Points_Map.drop(['code_x', 'code_y'], axis=1, inplace=True)
FIND_Regions_Points_Map.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN,33.0000,65.0
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN,41.0000,20.0
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,NaN,NaN,28.0000,3.0
3,American Samoa,"{""type"":""MultiPolygon"",""coordinates"":[[[[-170....",Test,– Increase diagnostic testing capacity,NaN,0 NGS facilities or equivalent,0,NaN,NaN,NaN,-14.3333,-170.0
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,NaN,NaN,42.5000,1.6


In [97]:
url = 'https://raw.githubusercontent.com/PandemicPreventionInstitute/NGS-Capacity-map/main/data/FIND_df_metadata.csv'
FIND_metadata = pd.read_csv (url)
FIND_metadata.head()

,country,access_tools,workforce_strength,enchance_data,max_connect,future_emerg
0,All,4,6,7,1,0
1,Africa,1,2,1,0,1
2,Cameroon,1,3,2,0,0
3,Brazil,6,9,8,5,5
4,Guinea,0,2,0,0,0


In [98]:
#merge together dataset
FIND_Projects_Org_Map = FIND_Regions_Points_Map.merge(FIND_metadata, how = 'left', on = 'country')
FIND_Projects_Org_Map.head(3)

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN,33.0,65.0,NaN,NaN,NaN,NaN,NaN
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN,41.0,20.0,NaN,NaN,NaN,NaN,NaN
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,NaN,NaN,28.0,3.0,NaN,NaN,NaN,NaN,NaN


In [99]:
#Explore Cameroon data
Cam_data = FIND_Projects_Org_Map.loc[(FIND_Projects_Org_Map['country'] == 'Cameroon')]
Cam_data.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg
34,Cameroon,"{""type"":""Polygon"",""coordinates"":[[[16.189823,2...",Leverage,– Leverage existing NGS capacity,41.15,1-3 NGS facilities or equivalent,211,Lower middle income,5.0,3.0,6.0,12.0,1.0,3.0,2.0,0.0,0.0


In [100]:
FIND_Projects_Org_Map.dtypes
FIND_Projects_Org_Map["access_tools"] = FIND_Projects_Org_Map["access_tools"].astype(float)
FIND_Projects_Org_Map["workforce_strength"] = FIND_Projects_Org_Map["workforce_strength"].astype(float)
FIND_Projects_Org_Map["enchance_data"] = FIND_Projects_Org_Map["enchance_data"].astype(float)
FIND_Projects_Org_Map["max_connect"] = FIND_Projects_Org_Map["max_connect"].astype(float)
FIND_Projects_Org_Map["future_emerg"] = FIND_Projects_Org_Map["future_emerg"].astype(float)
FIND_Projects_Org_Map.dtypes

country                       object
geometry                      object
archetype                     object
label                         object
max_new_tests_cap_avg        float64
sequencing_capacity_clean     object
total_sequences                int64
WB_eco                        object
N_projects                   float64
N_orgs                       float64
latitude                     float64
longitude                    float64
access_tools                 float64
workforce_strength           float64
enchance_data                float64
max_connect                  float64
future_emerg                 float64
dtype: object

In [101]:
def num (x):
    if x > 0:
        return '✓' 
    elif x <= 0:
        return 'X'
    else:
        return 'X'
    
FIND_Projects_Org_Map['access_tools'] = FIND_Projects_Org_Map['access_tools'].apply(lambda x : num (x))
FIND_Projects_Org_Map['workforce_strength'] = FIND_Projects_Org_Map['workforce_strength'].apply(lambda x : num (x))
FIND_Projects_Org_Map['enchance_data'] = FIND_Projects_Org_Map['enchance_data'].apply(lambda x : num (x))
FIND_Projects_Org_Map['max_connect'] = FIND_Projects_Org_Map['max_connect'].apply(lambda x : num (x))
FIND_Projects_Org_Map['future_emerg'] = FIND_Projects_Org_Map['future_emerg'].apply(lambda x : num (x))
FIND_Projects_Org_Map.head(2)

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN,33.0,65.0,X,X,X,X,X
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN,41.0,20.0,X,X,X,X,X


In [102]:
#make high income and LMIC dfs

#High Income df
HIC_df = FIND_Projects_Org_Map.loc[(FIND_Projects_Org_Map['archetype'] == 'High Income*')]
HIC_df.head()

#LMIC df
LMIC_df = FIND_Projects_Org_Map.loc[(FIND_Projects_Org_Map['archetype'] != 'High Income*')]
LMIC_df.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN,33.0000,65.0,X,X,X,X,X
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN,41.0000,20.0,X,X,X,X,X
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,NaN,NaN,28.0000,3.0,X,X,X,X,X
3,American Samoa,"{""type"":""MultiPolygon"",""coordinates"":[[[[-170....",Test,– Increase diagnostic testing capacity,NaN,0 NGS facilities or equivalent,0,NaN,NaN,NaN,-14.3333,-170.0,X,X,X,X,X
5,Angola,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.00...",Leverage,– Leverage existing NGS capacity,16.29,0 NGS facilities or equivalent,945,NaN,NaN,NaN,-12.5000,18.5,X,X,X,X,X


In [103]:
HIC_df.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,NaN,NaN,42.5000,1.6000,X,X,X,X,X
7,Antigua and Barbuda,"{""type"":""MultiPolygon"",""coordinates"":[[[[-61.8...",High Income*,NaN,131.09,High Income*,67,NaN,NaN,NaN,17.0500,-61.8000,X,X,X,X,X
10,Aruba,"{""type"":""Polygon"",""coordinates"":[[[-69.996938,...",High Income*,NaN,NaN,High Income*,2676,NaN,NaN,NaN,12.5000,-69.9667,X,X,X,X,X
11,Australia,"{""type"":""MultiPolygon"",""coordinates"":[[[[158.8...",High Income*,NaN,863.04,High Income*,34791,High income,1.0,1.0,-27.0000,133.0000,X,X,✓,X,X
12,Austria,"{""type"":""Polygon"",""coordinates"":[[[13.815725,4...",High Income*,NaN,5074.80,High Income*,59514,NaN,NaN,NaN,47.3333,13.3333,X,X,X,X,X


In [104]:
def num (x):
    if x == "✓":
        return 'HIC receiving partner support' 
    elif x == "X":
        return 'HIC not receiving partner support'
    else:
        return "HIC not receiving partner support"

#Delineate HIC and LMIC support...    
HIC_df['access_tools_support'] = HIC_df['access_tools'].apply(lambda x : num (x))
HIC_df['workforce_strength_support'] = HIC_df['workforce_strength'].apply(lambda x : num (x))
HIC_df['enchance_data_support'] = HIC_df['enchance_data'].apply(lambda x : num (x))
HIC_df['max_connect_support'] = HIC_df['max_connect'].apply(lambda x : num (x))
HIC_df['future_emerg_support'] = HIC_df['future_emerg'].apply(lambda x : num (x))
HIC_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,NaN,NaN,42.50,1.6,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support
7,Antigua and Barbuda,"{""type"":""MultiPolygon"",""coordinates"":[[[[-61.8...",High Income*,NaN,131.09,High Income*,67,NaN,NaN,NaN,17.05,-61.8,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support


In [116]:
#Fill in NAN values with 0
# NOTE: make sure to have NaN available for "Point" map, where you do not want 0's popping up for all countries
HIC_df['N_projects'] = HIC_df ['N_projects'].replace(np.inf, np.nan)
HIC_df['N_projects'].fillna(0, inplace = True)
HIC_df["N_projects"] = HIC_df["N_projects"].astype(int)

#filling in NaN with 0's for organization metric as well
HIC_df['N_orgs'] = HIC_df ['N_orgs'].replace(np.inf, np.nan)
HIC_df['N_orgs'].fillna(0, inplace = True)
HIC_df["N_orgs"] = HIC_df["N_orgs"].astype(int)

#confirming all reported metrics are integers and reviewing head of df
print (HIC_df.dtypes)
HIC_df.head(3)

country                        object
geometry                       object
archetype                      object
label                          object
max_new_tests_cap_avg         float64
sequencing_capacity_clean      object
total_sequences                 int64
WB_eco                         object
N_projects                      int64
N_orgs                          int64
latitude                      float64
longitude                     float64
access_tools                   object
workforce_strength             object
enchance_data                  object
max_connect                    object
future_emerg                   object
access_tools_support           object
workforce_strength_support     object
enchance_data_support          object
max_connect_support            object
future_emerg_support           object
Projects_Funded                 int64
support                        object
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,0,0,42.50,1.6000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
7,Antigua and Barbuda,"{""type"":""MultiPolygon"",""coordinates"":[[[[-61.8...",High Income*,NaN,131.09,High Income*,67,NaN,0,0,17.05,-61.8000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
10,Aruba,"{""type"":""Polygon"",""coordinates"":[[[-69.996938,...",High Income*,NaN,NaN,High Income*,2676,NaN,0,0,12.50,-69.9667,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support


In [117]:
def num (x):
    if x == 0:
        return 0 #Yes
    elif x > 0:
        return 1 #No

HIC_df['Projects_Funded'] = HIC_df['N_projects'].apply(lambda x : num (x))
HIC_df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,0,0,42.50,1.6000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
7,Antigua and Barbuda,"{""type"":""MultiPolygon"",""coordinates"":[[[[-61.8...",High Income*,NaN,131.09,High Income*,67,NaN,0,0,17.05,-61.8000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
10,Aruba,"{""type"":""Polygon"",""coordinates"":[[[-69.996938,...",High Income*,NaN,NaN,High Income*,2676,NaN,0,0,12.50,-69.9667,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support


In [118]:
#Fill in NAN values with 0
HIC_df['Projects_Funded'] = HIC_df ['Projects_Funded'].replace(np.inf, np.nan)
HIC_df['Projects_Funded'].fillna(0, inplace = True)
HIC_df["Projects_Funded"] = HIC_df["Projects_Funded"].astype(int)
print (HIC_df.dtypes)
HIC_df.head()

country                        object
geometry                       object
archetype                      object
label                          object
max_new_tests_cap_avg         float64
sequencing_capacity_clean      object
total_sequences                 int64
WB_eco                         object
N_projects                      int64
N_orgs                          int64
latitude                      float64
longitude                     float64
access_tools                   object
workforce_strength             object
enchance_data                  object
max_connect                    object
future_emerg                   object
access_tools_support           object
workforce_strength_support     object
enchance_data_support          object
max_connect_support            object
future_emerg_support           object
Projects_Funded                 int64
support                        object
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,0,0,42.5000,1.6000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
7,Antigua and Barbuda,"{""type"":""MultiPolygon"",""coordinates"":[[[[-61.8...",High Income*,NaN,131.09,High Income*,67,NaN,0,0,17.0500,-61.8000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
10,Aruba,"{""type"":""Polygon"",""coordinates"":[[[-69.996938,...",High Income*,NaN,NaN,High Income*,2676,NaN,0,0,12.5000,-69.9667,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
11,Australia,"{""type"":""MultiPolygon"",""coordinates"":[[[[158.8...",High Income*,NaN,863.04,High Income*,34791,High income,1,1,-27.0000,133.0000,X,X,✓,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC receiving partner support,HIC not receiving partner support,HIC not receiving partner support,1,HIC receiving partner support
12,Austria,"{""type"":""Polygon"",""coordinates"":[[[13.815725,4...",High Income*,NaN,5074.80,High Income*,59514,NaN,0,0,47.3333,13.3333,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support


In [107]:
def num (x):
    if x == 0.0: #No projects
        return "HIC not receiving partner support"
    elif x == 1.0: #Yes, projects
        return "HIC receiving partner support"

HIC_df['support'] = HIC_df['Projects_Funded'].apply(lambda x : num (x))
HIC_df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
4,Andorra,"{""type"":""Polygon"",""coordinates"":[[[1.429297,42...",High Income*,NaN,11029.48,High Income*,29,NaN,NaN,NaN,42.50,1.6000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
7,Antigua and Barbuda,"{""type"":""MultiPolygon"",""coordinates"":[[[[-61.8...",High Income*,NaN,131.09,High Income*,67,NaN,NaN,NaN,17.05,-61.8000,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support
10,Aruba,"{""type"":""Polygon"",""coordinates"":[[[-69.996938,...",High Income*,NaN,NaN,High Income*,2676,NaN,NaN,NaN,12.50,-69.9667,X,X,X,X,X,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,HIC not receiving partner support,0,HIC not receiving partner support


Data Processing and Tranformation of LMICs df

In [109]:
def num (x):
    if x == "✓":
        return 'LMIC receiving partner support' 
    elif x == "X":
        return 'LMIC not receiving partner support'
    else:
        return 'LMIC not receiving partner support'

#Delineate HIC and LMIC support...    
LMIC_df['access_tools_support'] = LMIC_df['access_tools'].apply(lambda x : num (x))
LMIC_df['workforce_strength_support'] = LMIC_df['workforce_strength'].apply(lambda x : num (x))
LMIC_df['enchance_data_support'] = LMIC_df['enchance_data'].apply(lambda x : num (x))
LMIC_df['max_connect_support'] = LMIC_df['max_connect'].apply(lambda x : num (x))
LMIC_df['future_emerg_support'] = LMIC_df['future_emerg'].apply(lambda x : num (x))
LMIC_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,NaN,NaN,33.0,65.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,NaN,NaN,41.0,20.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support


In [113]:
#Fill in NAN values with 0
# NOTE: make sure to have NaN available for "Point" map, where you do not want 0's popping up for all countries
LMIC_df['N_projects'] = LMIC_df ['N_projects'].replace(np.inf, np.nan)
LMIC_df['N_projects'].fillna(0, inplace = True)
LMIC_df["N_projects"] = LMIC_df["N_projects"].astype(int)

#filling in NaN with 0's for organization metric as well
LMIC_df['N_orgs'] = LMIC_df ['N_orgs'].replace(np.inf, np.nan)
LMIC_df['N_orgs'].fillna(0, inplace = True)
LMIC_df["N_orgs"] = LMIC_df["N_orgs"].astype(int)

#confirming all reported metrics are integers and reviewing head of df
print (LMIC_df.dtypes)
LMIC_df.head(3)

country                        object
geometry                       object
archetype                      object
label                          object
max_new_tests_cap_avg         float64
sequencing_capacity_clean      object
total_sequences                 int64
WB_eco                         object
N_projects                      int64
N_orgs                          int64
latitude                      float64
longitude                     float64
access_tools                   object
workforce_strength             object
enchance_data                  object
max_connect                    object
future_emerg                   object
access_tools_support           object
workforce_strength_support     object
enchance_data_support          object
max_connect_support            object
future_emerg_support           object
Projects_Funded               float64
support                        object
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,0,0,33.0,65.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,NaN,None
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,0,0,41.0,20.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,NaN,None
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,0,0,28.0,3.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,NaN,None


In [114]:
def num (x):
    if x == 0:
        return 0 #Yes, projects
    elif x > 0:
        return 1 #No, projects

LMIC_df['Projects_Funded'] = LMIC_df['N_projects'].apply(lambda x : num (x))
LMIC_df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,0,0,33.0,65.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,None
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,0,0,41.0,20.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,None
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,0,0,28.0,3.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,None


In [115]:
def num (x):
    if x == 0.0:
        return "LMIC not receiving partner support"
    elif x == 1.0:
        return "LMIC receiving partner support"

LMIC_df['support'] = LMIC_df['Projects_Funded'].apply(lambda x : num (x))
LMIC_df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
0,Afghanistan,"{""type"":""Polygon"",""coordinates"":[[[74.892307,3...",Connect,– Set-up sample referral networks or build NGS...,10.65,0 NGS facilities or equivalent,94,NaN,0,0,33.0,65.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support
1,Albania,"{""type"":""Polygon"",""coordinates"":[[[20.943662,4...",Connect,– Set-up sample referral networks or build NGS...,190.16,0 NGS facilities or equivalent,42,NaN,0,0,41.0,20.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,0,0,28.0,3.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support


Confirm df's are correct by testing country: Cameroon

In [119]:
#Exploring Cameroon to make sure this data makes sense
Cameroon_final_df = LMIC_df.loc[(LMIC_df['country'] == 'Cameroon')]
Cameroon_final_df.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
34,Cameroon,"{""type"":""Polygon"",""coordinates"":[[[16.189823,2...",Leverage,– Leverage existing NGS capacity,41.15,1-3 NGS facilities or equivalent,211,Lower middle income,5,3,6.0,12.0,✓,✓,✓,X,X,LMIC receiving partner support,LMIC receiving partner support,LMIC receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,1,LMIC receiving partner support


In [120]:
#Exploring Sudan to make sure this data makes sense
Sudan_final_df = LMIC_df.loc[(LMIC_df['country'] == 'Sudan')]
Sudan_final_df.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
200,Sudan,"{""type"":""MultiPolygon"",""coordinates"":[[[[31.49...",Test,– Increase diagnostic testing capacity,5.39,0 NGS facilities or equivalent,111,Low income,1,1,15.0,30.0,X,✓,✓,X,X,LMIC not receiving partner support,LMIC receiving partner support,LMIC receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,1,LMIC receiving partner support


In [121]:
#Exploring Sudan to make sure this data makes sense
Algeria_final_df = LMIC_df.loc[(LMIC_df['country'] == 'Algeria')]
Algeria_final_df.head()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
2,Algeria,"{""type"":""Polygon"",""coordinates"":[[[-8.681222,2...",Connect,– Set-up sample referral networks or build NGS...,4.46,0 NGS facilities or equivalent,72,NaN,0,0,28.0,3.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support


Concatenate HIC & LMICs together to make the joined/final df for the FIND partner maps

In [122]:
#Concat the df's to stack them
# concatenating df1 and df2 along rows
FIND_partner_maps = pd.concat([HIC_df, LMIC_df], axis=0)
FIND_partner_maps.tail()

,country,geometry,archetype,label,max_new_tests_cap_avg,sequencing_capacity_clean,total_sequences,WB_eco,N_projects,N_orgs,latitude,longitude,access_tools,workforce_strength,enchance_data,max_connect,future_emerg,access_tools_support,workforce_strength_support,enchance_data_support,max_connect_support,future_emerg_support,Projects_Funded,support
233,West Bank and Gaza,"{""type"":""MultiPolygon"",""coordinates"":[[[[34.48...",Connect,– Set-up sample referral networks or build NGS...,242.23,0 NGS facilities or equivalent,132,NaN,0,0,NaN,NaN,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support
234,Western Sahara,"{""type"":""MultiPolygon"",""coordinates"":[[[[-10.6...",Test,– Increase diagnostic testing capacity,9.15,0 NGS facilities or equivalent,0,NaN,0,0,24.5,-13.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support
235,Yemen,"{""type"":""MultiPolygon"",""coordinates"":[[[[51.97...",Connect,– Set-up sample referral networks or build NGS...,NaN,0 NGS facilities or equivalent,0,NaN,0,0,15.0,48.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support
236,Zambia,"{""type"":""Polygon"",""coordinates"":[[[32.920863,-...",Strengthen,– Build additional NGS capacity for scale-up,55.74,1-3 NGS facilities or equivalent,970,Lower middle income,4,4,-15.0,30.0,✓,✓,✓,X,X,LMIC receiving partner support,LMIC receiving partner support,LMIC receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,1,LMIC receiving partner support
237,Zimbabwe,"{""type"":""Polygon"",""coordinates"":[[[31.288922,-...",Leverage,– Leverage existing NGS capacity,77.95,0 NGS facilities or equivalent,657,Lower middle income,0,0,-20.0,30.0,X,X,X,X,X,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,LMIC not receiving partner support,0,LMIC not receiving partner support


In [123]:
#assessing shape (number of df [rows, columns])
FIND_partner_maps.shape
#confirm that we have the right amount of countries listed --238

(238, 24)

In [124]:
# Export dataset to .csv
from google.colab import files
FIND_partner_maps.to_csv('FIND_partner_maps.csv') 
files.download('FIND_partner_maps.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Making different datasets to visualize different FIND intervention support, by country and economic class